In [5]:
import tensorflow as tf

### Tensor
텐서는 다차원 배열입니다. numpy의 배열과 유사하게 tf.Tensor객체에는 데이터 유형과 모양이 있습니다. tf.Tensor는 GPU와 같은 가속기 메모리에 상주할 수 있습니다. Tensorflow는 tf.Tensor를 소비하고 생성하는 풍부한 연산 라이브러리를 제공합니다.(tf.math.add(덧셈), tf.linalg.matmul(행렬곱), tf.linalg.inv(역행렬) 등). 이러한 연산은 기본 python 유형을 자동으로 변환합니다.

In [6]:
print(tf.math.add(1,2))
print(tf.math.add([1,2],[3,4]))
print(tf.math.square(5))
print(tf.math.reduce_sum([1,2,3]))
print(tf.math.reduce_sum([[1,2,3],[1,2,5]]))
print(tf.math.square(2) + tf.math.square(3))

tf.Tensor(3, shape=(), dtype=int32)
tf.Tensor([4 6], shape=(2,), dtype=int32)
tf.Tensor(25, shape=(), dtype=int32)
tf.Tensor(6, shape=(), dtype=int32)
tf.Tensor(14, shape=(), dtype=int32)
tf.Tensor(13, shape=(), dtype=int32)


In [7]:
x = tf.linalg.matmul([[2]],[[2,3]])
print(x)
print(x.shape)
print(x.dtype)

tf.Tensor([[4 6]], shape=(1, 2), dtype=int32)
(1, 2)
<dtype: 'int32'>


### tf.Tensor의 장점
 - 텐서플로 연산은 자동으로 numpy배열을 텐서로 변환합니다.
 - numpy연산은 자동으로 텐서를 numpy배열로 변환합니다.
 - 텐서는 .numpy() 메서드를 통해 numpy배열로 변환이 가능합니다. 그러나 tf.Tensor는 GPU 메모리에 저장될 수 있고, numpy 배열은 항상 호스트 메모리에 저장되므로, 이러한 변환이 항상 가능한 것은 아닙니다. 따라서 gpu에서 호스트 메모리로 복사가 필요합니다.

In [11]:
import numpy as np

ndarray = np.ones([3,3])
print('ndarray : \n',ndarray)
tensor = tf.math.multiply(ndarray, 42)
print('tensor : \n',tensor)
# 자동으로 텐서가 넘파이 배열화 되어 연산됨을 확인해본다.
print('\n',np.add(tensor,1))
# 매서드를 통한 넘파이와 텐서간의 변화를 확인해본다.
print('\n',tensor.numpy())

ndarray : 
 [[1. 1. 1.]
 [1. 1. 1.]
 [1. 1. 1.]]
tensor : 
 tf.Tensor(
[[42. 42. 42.]
 [42. 42. 42.]
 [42. 42. 42.]], shape=(3, 3), dtype=float64)

 [[43. 43. 43.]
 [43. 43. 43.]
 [43. 43. 43.]]

 [[42. 42. 42.]
 [42. 42. 42.]
 [42. 42. 42.]]


## GPU 가속
 대부분의 텐서 연산은 GPU를 사용하여 가속한다. 따로 코드를 명시하지 않아도 tensorflow는 연산을 위해 CPU or GPU의 사용을 자동으로 결정한다. 필요시 둘 사이의 메모리에서 복사되기도 합니다. 연산에 의해 생성된 텐서는 연산이 실행된 장치의 메모리에 의해 실행됩니다.
 <br>예시는 아래와 같습니다

In [17]:
x = tf.random.uniform([3,3])
print(x)

print('GPU장치가 있는지 확인해본다.')
print(tf.config.list_physical_devices())

tf.Tensor(
[[0.98220205 0.75512254 0.70742786]
 [0.07006562 0.38545406 0.47248185]
 [0.16607213 0.7613646  0.6220701 ]], shape=(3, 3), dtype=float32)
GPU장치가 있는지 확인해본다.
[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]


### 위의 방식으로 확인해보면 GPU가 없음을 알 수 있다. 필자는 구글의 코랩 유료버전을 사용중이므로 TPU를 이용할 것이며 TPU에 대한 확인과 사용방법은 아래와 같다.

In [21]:
import os
x = tf.random.uniform([3,3])
print(x)

print('TPU장치가 있는지 확인해본다.')
print(os.environ['COLAB_TPU_ADDR'])
# tpu이름을 통해 해당 장치를 지정하여 사용이 가능한데 나의 tpu이름은 아래와 같이 사용하면 된다.
tpu_name = 'grpc://'+os.environ['COLAB_TPU_ADDR']
print('tpu_name : ',tpu_name)

tf.Tensor(
[[0.31053042 0.52364016 0.25591218]
 [0.02147949 0.14401782 0.20363021]
 [0.4052142  0.75381637 0.71540976]], shape=(3, 3), dtype=float32)
TPU장치가 있는지 확인해본다.
10.1.19.202:8470
tpu_name :  grpc://10.1.19.202:8470


TPU와 CPU의 성능 차이를 비교해 보겠다.

In [ ]:
# tpu 사용준비